In [128]:
# Properties 

In [129]:
import glob, openpyxl, pandas as pd, numpy as np, xlrd, statistics as stat
from datetime import datetime
from datetime import datetime, timedelta

In [130]:
# Function to get overall time interval for each patient
def get_overall_time(dataframe):
    
    #Get list of rows for a single patient
    row_per_patient = get_row_patient(dataframe)
    row_number = get_start_row_number(row_per_patient)
    
    # Create new list
    overall_time_list = []
    
    #Get overall time
    for patient in range(0,len(row_per_patient)):
        #print("patient number is")
        #print(patient)
        start_time = get_start_time(dataframe, row_number[patient])
        delta_start = timedelta(minutes=start_time.minute, seconds=start_time.second)
        if (patient == (len(row_per_patient)-1)):
            #print("if")
            #print(patient)
            end_time = get_end_time(dataframe, (len(dataframe)-1))
        else:
            #print("else")
            #print(patient)
            end_time = get_end_time(dataframe, (row_number[patient + 1]-1))
        delta_end = timedelta(minutes=end_time.minute, seconds=end_time.second)
        delta_total = delta_end - delta_start
        overall_time_list.append(delta_total)
    
    return overall_time_list

In [131]:
def get_start_time(dataframe, row_number):
    
    start_time = datetime.strptime(dataframe['Time (min)'].values[row_number],"%M:%S")
    
    return start_time

In [132]:
def get_end_time(dataframe, row_number):
    
    end_time = datetime.strptime(dataframe['Time (min)'].values[row_number],"%M:%S")
    
    return end_time

In [133]:
def get_row_patient(dataframe):
    
    # patient ID column
    df_patientID = df_column_to_list(dataframe, 0)
    
    # List to save row counts for single patient
    row_count = []
    
    # Count the rows
    patient_num = 1
    row = 0
    while (patient_num < (dataframe.iat[(len(df_patientID)-1), 0]+1)):
        count = 0
        for row in range(row, len(df_patientID)):
            if (df_patientID[row] == patient_num):
                count = count + 1
            else: 
                break
        row_count.append(count)
        patient_num = patient_num + 1
        
    return row_count

In [134]:
def get_start_row_number(row_count):
    
    # create a new list
    row_count_new = []
    
    for index in range(0, len(row_count)):
        if (index == 0):
            #print("0 in index = 0")
            value = 0
            row_count_new.append(value)
        elif (index == 1):
            #print("1 in index = 1")
            value = 0 + row_count[0]
            #print(value)
            row_count_new.append(value)
            #print(row_count_new)
        else:
            #print(index)
            value = row_count[index-1] + row_count_new[index - 1]
            #print(value)
            row_count_new.append(value)
            #print(row_count_new)
    
    return row_count_new
    
    

In [135]:
def get_yellow_row_start(dataframe, start_row_count):
    
    # create a new list to save the yellow start row number
    yellow_start_row = []
    
    for index in range(0, len(start_row_count)):
        if (index == 0):
            value = dataframe.iat[index, 19] - 1
            yellow_start_row.append(value)
        else:
            value = dataframe.iat[start_row_count[index], 19] - 1
            start_row = start_row_count[index]
            value = value + start_row
            yellow_start_row.append(value)
    
    return yellow_start_row
            

In [136]:
def get_yellow_row_end(dataframe, start_row_count):
    
    # 
    yellow_end_row = []
    
    for index in range(0, len(start_row_count)):
        if (index == 0):
            value = dataframe.iat[index, 20] - 1
            yellow_end_row.append(value)
        else:
            value = dataframe.iat[start_row_count[index], 20] - 1
            start_row = start_row_count[index]
            value = value + start_row
            yellow_end_row.append(value)
        
    return yellow_end_row

In [137]:
def yellow_overall_time(dataframe, yellow_start_row, yellow_end_row):
    
    #
    yellow_overall = []
    
    #Get overall time
    for patient in range(0,len(yellow_start_row)):
        #print("patient number is")
        #print(patient)
        start_time = get_start_time(df, yellow_start_row[patient])
        delta_start = timedelta(minutes=start_time.minute, seconds=start_time.second)
        end_time = get_end_time(df, yellow_end_row[patient])
        delta_end = timedelta(minutes=end_time.minute, seconds=end_time.second)
        delta_total = delta_end - delta_start
        yellow_overall.append(delta_total)
    
    return yellow_overall

In [138]:
def append_Multi_Item(data, itemList):
    #print("start append_multi_item")
    for i in range(0,(len(itemList))):
        data.append(itemList[i])
    #print("end append_multi_item")
    return data

In [139]:
def df_column_to_list(dataframe, column_number):

    # Create a new list
    data_in_column = []
    
    # get data
    for row in range(0, len(dataframe.index)):
        value = dataframe.iat[row,column_number]
        data_in_column.append(value)
    
    return data_in_column

In [140]:
def calc_mean_VO2(dataframe, start_row_count):
    
    # Create a new list
    mean_VO2 = []
    
    # get data of VO2
    for index in range(0, len(start_row_count)):
        #print(index)
        data_list = []
        if (index == len(start_row_count)-1):
            for row in range(start_row_count[index], len(dataframe)-1):
                value = dataframe.iat[row, 10]
                data_list.append(value)
            mean = stat.mean(data_list)
            mean_VO2.append(mean)
        else:
            for row in range(start_row_count[index], start_row_count[index+1]-1):
                value = dataframe.iat[row, 10]
                data_list.append(value)
            mean = stat.mean(data_list)
            mean_VO2.append(mean)
    
    return mean_VO2
        

In [141]:
def calc_mean_VO2_yellow(dataframe,start_row_count, yellow_start_row, yellow_end_row):
    
    # Create a new List
    mean_VO2_yellow = []
    
    # get data of VO2
    for index in range(0, len(start_row_count)):
        data_list = []
        for row in range(yellow_start_row[index], yellow_end_row[index]+1):
            value = dataframe.iat[row, 10]
            data_list.append(value)
        mean = stat.mean(data_list)
        mean_VO2_yellow.append(mean)
    
    return mean_VO2_yellow

In [142]:
def calc_mean_VCO2(dataframe, start_row_count):
    
     # Create a new list
    mean_VCO2 = []
    
    # get data of VCO2
    for index in range(0, len(start_row_count)):
        #print(index)
        data_list = []
        if (index == len(start_row_count)-1):
            for row in range(start_row_count[index], len(dataframe)-1):
                value = dataframe.iat[row, 12]
                data_list.append(value)
            mean = stat.mean(data_list)
            mean_VCO2.append(mean)
        else:
            for row in range(start_row_count[index], start_row_count[index+1]-1):
                value = dataframe.iat[row, 12]
                data_list.append(value)
            mean = stat.mean(data_list)
            mean_VCO2.append(mean)
    
    return mean_VCO2

In [143]:
def calc_mean_VCO2_yellow(dataframe,start_row_count, yellow_start_row, yellow_end_row):
    
    # Create a new List
    mean_VCO2_yellow = []
    
    # get data of VCO2
    for index in range(0, len(start_row_count)):
        data_list = []
        for row in range(yellow_start_row[index], yellow_end_row[index]+1):
            value = dataframe.iat[row, 12]
            data_list.append(value)
        mean = stat.mean(data_list)
        mean_VCO2_yellow.append(mean)
    
    return mean_VCO2_yellow

In [144]:
def calc_mean_VE(dataframe, start_row_count):
    
    # Create a new list
    mean_VE = []
    
    # get data of VCO2
    for index in range(0, len(start_row_count)):
        #print(index)
        data_list = []
        if (index == len(start_row_count)-1):
            for row in range(start_row_count[index], len(dataframe)-1):
                value = dataframe.iat[row, 13]
                data_list.append(value)
            mean = stat.mean(data_list)
            mean_VE.append(mean)
        else:
            for row in range(start_row_count[index], start_row_count[index+1]-1):
                value = dataframe.iat[row, 13]
                data_list.append(value)
            mean = stat.mean(data_list)
            mean_VE.append(mean)
    
    return mean_VE

In [145]:
def calc_mean_VE_yellow(dataframe,start_row_count, yellow_start_row, yellow_end_row):
    
    # Create a new List
    mean_VE_yellow = []
    
    # get data of VCO2
    for index in range(0, len(start_row_count)):
        data_list = []
        for row in range(yellow_start_row[index], yellow_end_row[index]+1):
            value = dataframe.iat[row, 13]
            data_list.append(value)
        mean = stat.mean(data_list)
        mean_VE_yellow.append(mean)
    
    return mean_VE_yellow

In [146]:
def calc_mean_BPM(dataframe, start_row_count):
    
    # Create a new list
    mean_BPM = []
    
    # get data of VCO2
    for index in range(0, len(start_row_count)):
        #print(index)
        data_list = []
        if (index == len(start_row_count)-1):
            for row in range(start_row_count[index], len(dataframe)-1):
                value = dataframe.iat[row, 15]
                data_list.append(value)
            mean = stat.mean(data_list)
            mean_BPM.append(mean)
        else:
            for row in range(start_row_count[index], start_row_count[index+1]-1):
                value = dataframe.iat[row, 15]
                data_list.append(value)
            mean = stat.mean(data_list)
            mean_BPM.append(mean)
    
    return mean_BPM

In [147]:
def calc_mean_BPM_yellow(dataframe,start_row_count, yellow_start_row, yellow_end_row):
    
    # Create a new List
    mean_VE_BPM = []
    
    # get data of VCO2
    for index in range(0, len(start_row_count)):
        data_list = []
        for row in range(yellow_start_row[index], yellow_end_row[index]+1):
            value = dataframe.iat[row, 15]
            data_list.append(value)
        mean = stat.mean(data_list)
        mean_VE_BPM.append(mean)
    
    return mean_VE_BPM

In [148]:
def calc_var_VO2(dataframe, start_row_count):
    
     # Create a new list
    var_VO2 = []
    
    # get data of VO2
    for index in range(0, len(start_row_count)):
        #print(index)
        data_list = []
        if (index == len(start_row_count)-1):
            for row in range(start_row_count[index], len(dataframe)-1):
                value = dataframe.iat[row, 10]
                data_list.append(value)
            var = stat.variance(data_list)
            var_VO2.append(var)
        else:
            for row in range(start_row_count[index], start_row_count[index+1]-1):
                value = dataframe.iat[row, 10]
                data_list.append(value)
            var = stat.variance(data_list)
            var_VO2.append(var)
    
    return var_VO2

In [149]:
def calc_var_VCO2(dataframe, start_row_count):
    
    # Create a new list
    var_VCO2 = []
    
    # get data of VO2
    for index in range(0, len(start_row_count)):
        #print(index)
        data_list = []
        if (index == len(start_row_count)-1):
            for row in range(start_row_count[index], len(dataframe)-1):
                value = dataframe.iat[row, 12]
                data_list.append(value)
            var = stat.variance(data_list)
            var_VCO2.append(var)
        else:
            for row in range(start_row_count[index], start_row_count[index+1]-1):
                value = dataframe.iat[row, 12]
                data_list.append(value)
            var = stat.variance(data_list)
            var_VCO2.append(var)
    
    return var_VCO2

In [150]:
def calc_var_VE(dataframe, start_row_count):
    
    # Create a new list
    var_VE = []
    
    # get data of VO2
    for index in range(0, len(start_row_count)):
        #print(index)
        data_list = []
        if (index == len(start_row_count)-1):
            for row in range(start_row_count[index], len(dataframe)-1):
                value = dataframe.iat[row, 13]
                data_list.append(value)
            var = stat.variance(data_list)
            var_VE.append(var)
        else:
            for row in range(start_row_count[index], start_row_count[index+1]-1):
                value = dataframe.iat[row, 13]
                data_list.append(value)
            var = stat.variance(data_list)
            var_VE.append(var)
    
    return var_VE

In [151]:
def calc_var_BPM(dataframe, start_row_count):
    
    # Create a new list
    var_BPM = []
    
    # get data of VO2
    for index in range(0, len(start_row_count)):
        #print(index)
        data_list = []
        if (index == len(start_row_count)-1):
            for row in range(start_row_count[index], len(dataframe)-1):
                value = dataframe.iat[row, 15]
                data_list.append(value)
            var = stat.variance(data_list)
            var_BPM.append(var)
        else:
            for row in range(start_row_count[index], start_row_count[index+1]-1):
                value = dataframe.iat[row, 15]
                data_list.append(value)
            var = stat.variance(data_list)
            var_BPM.append(var)
    
    return var_BPM

In [152]:
fname = 'P:\\Metabolic Test\\RMR\\data\\data_all.xlsx'
df = pd.read_excel(fname)

In [153]:
df

,Patient ID,Age,Sex,Height (in),Height (cm),Weight (Ib),Weight (kg),Time (min),VO2/kg STPD (ml/kg/m),VO2 STPD (L/min),...,VCO2 STPD (L/min),(ml/min).1,VE STPD (L/min),RQ,RR (BPM),VE/VO2 (BT/ST),METS,TM SPD(mph),Start,End
0,1,46,F,67.000000,170.179993,186.380997,84.718636,0:18,4.205394,0.356275,...,0.247425,247.424841,7.473200,0.694477,12.834225,25.997837,1.201541,0.0,40,58
0,1,46,F,67.000000,170.179993,186.380997,84.718636,0:30,3.374727,0.285902,...,0.201463,201.463372,5.990118,0.704658,10.335917,25.967733,0.964208,0.0,40,58
0,1,46,F,67.000000,170.179993,186.380997,84.718636,0:49,3.794683,0.321480,...,0.227283,227.283075,6.813195,0.706989,9.424084,26.267130,1.084195,0.0,40,58
0,1,46,F,67.000000,170.179993,186.380997,84.718636,1:03,3.169709,0.268533,...,0.193977,193.977356,5.767123,0.722358,8.797654,26.618103,0.905631,0.0,40,58
0,1,46,F,67.000000,170.179993,186.380997,84.718636,1:20,3.762461,0.318751,...,0.238344,238.344163,6.795503,0.747745,6.737788,26.423296,1.074989,0.0,40,58
0,1,46,F,67.000000,170.179993,186.380997,84.718636,1:35,3.324426,0.281641,...,0.211946,211.945727,5.899158,0.752539,8.102633,25.960358,0.949836,0.0,40,58
0,1,46,F,67.000000,170.179993,186.380997,84.718636,1:46,3.010240,0.255023,...,0.191315,191.314548,5.261397,0.750184,10.978957,25.570383,0.860069,0.0,40,58
0,1,46,F,67.000000,170.179993,186.380997,84.718636,2:08,3.527878,0.298877,...,0.225742,225.742131,6.074594,0.755301,8.272058,25.190744,1.007965,0.0,40,58
0,1,46,F,67.000000,170.179993,186.380997,84.718636,2:16,3.503185,0.296785,...,0.222011,222.010523,6.056265,0.748051,7.509387,25.291759,1.000910,0.0,40,58
0,1,46,F,67.000000,170.179993,186.380997,84.718636,2:32,3.069382,0.260034,...,0.189974,189.974263,5.366534,0.730575,7.532957,25.578804,0.876966,0.0,40,58


In [154]:
row_patient = get_row_patient(df)

[190,
 148,
 148,
 164,
 120,
 103,
 144,
 119,
 110,
 132,
 116,
 128,
 123,
 108,
 200,
 96,
 122,
 122,
 122,
 125,
 138,
 104,
 103,
 106,
 117,
 128,
 118,
 118,
 164,
 120,
 122,
 170,
 135,
 128,
 121,
 121,
 118,
 90,
 136,
 120,
 138,
 134,
 123,
 120,
 126,
 49,
 142,
 119,
 129,
 127,
 115,
 116,
 114,
 129,
 144,
 164,
 137,
 120,
 122,
 150,
 175,
 122,
 120,
 121,
 141,
 186,
 123,
 123,
 120,
 127,
 113,
 140,
 126,
 122]

In [155]:
start_row_patient = get_start_row_number(row_patient)
start_row_patient

[0,
 190,
 338,
 486,
 650,
 770,
 873,
 1017,
 1136,
 1246,
 1378,
 1494,
 1622,
 1745,
 1853,
 2053,
 2149,
 2271,
 2393,
 2515,
 2640,
 2778,
 2882,
 2985,
 3091,
 3208,
 3336,
 3454,
 3572,
 3736,
 3856,
 3978,
 4148,
 4283,
 4411,
 4532,
 4653,
 4771,
 4861,
 4997,
 5117,
 5255,
 5389,
 5512,
 5632,
 5758,
 5807,
 5949,
 6068,
 6197,
 6324,
 6439,
 6555,
 6669,
 6798,
 6942,
 7106,
 7243,
 7363,
 7485,
 7635,
 7810,
 7932,
 8052,
 8173,
 8314,
 8500,
 8623,
 8746,
 8866,
 8993,
 9106,
 9246,
 9372]

In [156]:
overall_time = get_overall_time(df)
overall_time

[datetime.timedelta(0, 2832),
 datetime.timedelta(0, 2205),
 datetime.timedelta(0, 2204),
 datetime.timedelta(0, 2439),
 datetime.timedelta(0, 1784),
 datetime.timedelta(0, 1530),
 datetime.timedelta(0, 2163),
 datetime.timedelta(0, 1763),
 datetime.timedelta(0, 1640),
 datetime.timedelta(0, 1966),
 datetime.timedelta(0, 1911),
 datetime.timedelta(0, 1904),
 datetime.timedelta(0, 1824),
 datetime.timedelta(0, 1607),
 datetime.timedelta(0, 2985),
 datetime.timedelta(0, 1418),
 datetime.timedelta(0, 1815),
 datetime.timedelta(0, 1812),
 datetime.timedelta(0, 1831),
 datetime.timedelta(0, 1855),
 datetime.timedelta(0, 2047),
 datetime.timedelta(0, 1545),
 datetime.timedelta(0, 1576),
 datetime.timedelta(0, 1567),
 datetime.timedelta(0, 1736),
 datetime.timedelta(0, 1902),
 datetime.timedelta(0, 1752),
 datetime.timedelta(0, 1758),
 datetime.timedelta(0, 2438),
 datetime.timedelta(0, 1788),
 datetime.timedelta(0, 1814),
 datetime.timedelta(0, 2593),
 datetime.timedelta(0, 2004),
 datetime.

In [157]:
yellow_start = get_yellow_row_start(df, start_row_patient)
yellow_start

[39,
 240,
 350,
 571,
 669,
 836,
 949,
 1096,
 1165,
 1290,
 1402,
 1536,
 1662,
 1805,
 1912,
 2073,
 2188,
 2311,
 2425,
 2521,
 2687,
 2807,
 2921,
 3041,
 3121,
 3227,
 3399,
 3473,
 3701,
 3811,
 3885,
 4103,
 4208,
 4292,
 4462,
 4576,
 4697,
 4786,
 4876,
 5005,
 5122,
 5346,
 5408,
 5570,
 5667,
 5779,
 5824,
 6024,
 6081,
 6220,
 6364,
 6471,
 6584,
 6746,
 6843,
 7041,
 7145,
 7290,
 7414,
 7564,
 7656,
 7818,
 7955,
 8075,
 8209,
 8327,
 8520,
 8705,
 8806,
 8923,
 9041,
 9141,
 9309,
 9414]

In [158]:
yellow_end = get_yellow_row_end(df, start_row_patient)
yellow_end

[57,
 280,
 391,
 609,
 709,
 847,
 987,
 1134,
 1205,
 1330,
 1440,
 1576,
 1702,
 1845,
 1960,
 2113,
 2228,
 2351,
 2426,
 2560,
 2728,
 2847,
 2961,
 3081,
 3161,
 3267,
 3439,
 3513,
 3732,
 3855,
 3925,
 4142,
 4240,
 4316,
 4502,
 4616,
 4717,
 4823,
 4916,
 5045,
 5162,
 5386,
 5448,
 5611,
 5707,
 5790,
 5865,
 6064,
 6121,
 6260,
 6403,
 6511,
 6624,
 6786,
 6883,
 7081,
 7185,
 7332,
 7454,
 7607,
 7687,
 7852,
 7995,
 8115,
 8249,
 8357,
 8560,
 8745,
 8846,
 8963,
 9081,
 9181,
 9349,
 9454]

In [159]:
yellow_overall_time = yellow_overall_time(df, yellow_start, yellow_end)
yellow_overall_time

[datetime.timedelta(0, 272),
 datetime.timedelta(0, 601),
 datetime.timedelta(0, 617),
 datetime.timedelta(0, 571),
 datetime.timedelta(0, 599),
 datetime.timedelta(0, 164),
 datetime.timedelta(0, 592),
 datetime.timedelta(0, 569),
 datetime.timedelta(0, 601),
 datetime.timedelta(0, 603),
 datetime.timedelta(0, 747),
 datetime.timedelta(0, 599),
 datetime.timedelta(0, 604),
 datetime.timedelta(0, 597),
 datetime.timedelta(0, 723),
 datetime.timedelta(0, 599),
 datetime.timedelta(0, 602),
 datetime.timedelta(0, 598),
 datetime.timedelta(0, 10),
 datetime.timedelta(0, 583),
 datetime.timedelta(0, 617),
 datetime.timedelta(0, 598),
 datetime.timedelta(0, 595),
 datetime.timedelta(0, 602),
 datetime.timedelta(0, 600),
 datetime.timedelta(0, 462),
 datetime.timedelta(0, 597),
 datetime.timedelta(0, 600),
 datetime.timedelta(0, 438),
 datetime.timedelta(0, 663),
 datetime.timedelta(0, 603),
 datetime.timedelta(0, 603),
 datetime.timedelta(0, 481),
 datetime.timedelta(0, 362),
 datetime.timed

In [160]:
mean_VO2 = calc_mean_VO2(df, start_row_patient)
mean_VO2

[281.30877577753927,
 295.24316349808049,
 225.37991566722897,
 243.98876909463689,
 263.91163355663042,
 279.04296819778051,
 53.183159875598818,
 276.76682350999221,
 242.22879282651695,
 346.35768651621038,
 267.26569552784377,
 250.09209856273623,
 273.67686724565067,
 270.20980320244195,
 242.1656370762006,
 218.62132204206367,
 209.4071466814388,
 266.36354982360336,
 408.67163573414825,
 254.98523274737019,
 285.43230325636205,
 246.25193841249046,
 315.97584588270564,
 371.67007208225272,
 239.73796388198582,
 192.04475666124992,
 239.60508508050546,
 364.34796923755579,
 283.62992794418631,
 244.06113607041971,
 395.03011600045136,
 274.98199967061277,
 205.80547336322158,
 239.62042592172554,
 235.81218669811886,
 349.61885561545688,
 286.02347796798773,
 173.28722733125258,
 259.28499566184149,
 225.22827062536686,
 319.84473503854156,
 243.50387799112417,
 255.66810235136847,
 308.10733991009846,
 314.53575360774994,
 319.12442110478878,
 226.83017901706356,
 221.0883688560

In [161]:
mean_VCO2 = calc_mean_VCO2(df, start_row_patient)
mean_VCO2

[194.46650604722362,
 227.22516903260933,
 162.83871831537104,
 188.58625954645547,
 205.18059965943095,
 200.08561818623076,
 40.468306632360793,
 211.78964007708987,
 186.00998200271104,
 288.18338283316325,
 221.13349547852641,
 197.26638114593163,
 219.45337816828581,
 207.21966341555671,
 177.14895236881534,
 173.74691712228875,
 171.23132402246648,
 188.28717551448128,
 373.44992444042333,
 203.95203247185677,
 238.00152554744525,
 182.22811796422147,
 255.14134063440207,
 272.8301118349745,
 159.37332237332035,
 136.71799408873233,
 187.36908935074109,
 305.07715886984124,
 232.17698067792355,
 160.89115028872214,
 308.54242042569092,
 202.9013301317508,
 152.17636664634321,
 192.4030746765963,
 186.62558173139891,
 276.27433277666569,
 220.24513118796878,
 136.37839492117422,
 195.21310528119409,
 173.6509489881642,
 262.96740280885768,
 183.42373469718416,
 191.95507185869531,
 231.43633974700415,
 252.49593663215637,
 246.28954473882922,
 151.4351414450517,
 171.0409009355609

In [162]:
mean_VE = calc_mean_VE(df, start_row_patient)
mean_VE

[6.0794036047799249,
 7.5286517889321258,
 5.3829834736934323,
 5.6200853798287049,
 6.932525991391735,
 6.3105939603319356,
 1.6194592351680037,
 6.1860786130872834,
 6.6602394799573705,
 8.6288696345482148,
 6.2391366937886108,
 5.8758684537542152,
 6.282996109274567,
 7.395399459054536,
 5.8363316334671707,
 4.905605042608161,
 5.5395053634958815,
 6.7565863979749441,
 14.261901207206666,
 6.5631140393595544,
 7.1935320437956198,
 6.1635558952405614,
 7.3544751756331497,
 8.3045109939007524,
 6.235558067930155,
 4.951051030572005,
 7.1485909559787846,
 9.0777784983317051,
 7.4317117673487747,
 6.0589045536618276,
 10.167614298418533,
 4.8231423359650831,
 5.1617413029741881,
 5.7329194095191056,
 6.5962411602338156,
 7.9439496338367483,
 6.7275114303980112,
 4.9983583648553056,
 6.4232323028423171,
 5.9672857633157941,
 7.762538081538068,
 5.4507527969924805,
 5.1924103859995245,
 7.7081510760203127,
 7.7354168777465819,
 7.6043378214041395,
 5.2293372999691794,
 5.1987517607414118,

In [163]:
mean_BPM = calc_mean_BPM(df, start_row_patient)
mean_BPM

[10.995742754961448,
 13.144402231488909,
 11.624660323266266,
 11.626628337462256,
 16.52877503483235,
 13.358915048487049,
 6.7673718695873992,
 11.842345601421295,
 14.836986174277209,
 14.949718981298778,
 6.2411260045093035,
 10.376152139949047,
 7.8995826166184226,
 17.939039872071451,
 13.672537645502905,
 12.877134664435136,
 11.888824892438148,
 16.058474454012785,
 8.0881301588263419,
 17.043966677881059,
 14.825072810218979,
 16.458353487033289,
 10.028417565074621,
 15.64324917339143,
 16.127076420290717,
 12.036624288934423,
 17.758664685436806,
 13.357971444089188,
 9.5028334658570071,
 16.79758657727923,
 8.5758291118401146,
 5.3859747669400537,
 13.683063926981456,
 9.9800943596156557,
 13.517419274648031,
 11.806440142790475,
 9.3624323290637381,
 11.929042580422388,
 12.732749151300505,
 11.119649286029718,
 11.753142739734511,
 10.171523789473685,
 6.6278122331275311,
 14.489668846130368,
 18.169931106567386,
 12.932770788669584,
 13.325215566242841,
 12.633960024785

In [164]:
var_VO2 = calc_var_VCO2(df, start_row_patient)
var_VO2

[477.54948559636449,
 1452.7275072419391,
 395.22770465570585,
 278.67830016079057,
 1188.6971330254476,
 1709.976429121738,
 156.47564072393817,
 221.74059714353055,
 1816.8717371342584,
 5019.0247784536659,
 3148.2138391703816,
 2355.7610678591973,
 2573.5871864307755,
 2110.3292727190533,
 1359.3653511681957,
 313.72218635218474,
 659.59921405992804,
 358.45405434256787,
 14817.825844692507,
 697.69994207376453,
 1295.3422824570632,
 940.64029560907068,
 1844.983029162242,
 2527.6437930134666,
 645.75620670855517,
 300.76600786911234,
 1081.0208931184136,
 2881.1766563555534,
 2618.5869859558202,
 672.00639691562151,
 4952.305725574598,
 1316.5235436211533,
 944.01092566377565,
 1851.59413206721,
 568.35542634101591,
 4608.9427474193953,
 2990.8698444158231,
 758.1414468595093,
 1023.0251852979361,
 1913.043080100445,
 4892.8417194690646,
 525.08537915910745,
 3474.2318387601272,
 848.63990212917327,
 853.34780598458508,
 1027.1791794940204,
 1176.6247746095025,
 1843.6259453331963,

In [165]:
mean_VO2_yellow = calc_mean_VO2_yellow(df, start_row_patient, yellow_start, yellow_end)
mean_VO2_yellow

[274.84433039238581,
 292.05107034706492,
 202.33831306298578,
 243.4204748043648,
 248.91098752254388,
 275.3535931309064,
 52.224681784327217,
 288.50233172759027,
 245.60610041385749,
 337.85270845017783,
 223.37401448152008,
 234.26068483329411,
 273.64921606168514,
 263.849593153814,
 234.01471060149527,
 209.42480971173543,
 199.92037190169819,
 262.69401282798958,
 289.76228088140499,
 243.80399212241178,
 273.1093771400906,
 236.83164577658596,
 305.63821625418785,
 379.35333317372857,
 229.37148555022907,
 182.50176056129169,
 246.17018677839428,
 373.60246443166972,
 253.69983352720743,
 249.05493855476377,
 379.29686903953547,
 259.58829857409,
 207.13939179073685,
 229.96072769165056,
 230.85816022826398,
 336.6993858319957,
 270.81444717588874,
 155.82547219176038,
 238.00333534798966,
 219.70257490146443,
 299.61438295317862,
 259.62019039363395,
 224.67228861116786,
 303.93926444507775,
 297.69449662871477,
 306.73886835575098,
 228.7399236645017,
 216.9081775880442,
 33

In [166]:
mean_VCO2_yellow = calc_mean_VCO2_yellow(df, start_row_patient, yellow_start, yellow_end)
mean_VCO2_yellow

[190.94308269651316,
 221.61643497827578,
 144.10439098165142,
 188.09378529206305,
 191.21575173808313,
 190.62039876977599,
 38.400008749121277,
 215.67640090600042,
 186.08985359712341,
 276.08253606935818,
 190.77673630836688,
 182.2237123439952,
 214.28675295376183,
 184.78649254979149,
 170.24530485576514,
 165.09183877851905,
 161.57693070609395,
 184.30472692338432,
 296.32285237312351,
 192.82127581536764,
 227.81121428571427,
 175.50138256898745,
 249.78931102810833,
 286.50676768000528,
 155.59325439900891,
 128.57002764940262,
 184.06834925820186,
 305.04515367310222,
 202.55235442891723,
 166.60300460126672,
 292.65963994875187,
 201.07658393681055,
 154.20713343403563,
 186.98334336280826,
 179.38901883799855,
 265.22754895977857,
 201.8869022528331,
 121.66413605997441,
 178.56266840202056,
 158.39838618185459,
 257.39028722774691,
 197.85421069075423,
 174.809546732321,
 228.65584031457001,
 242.80421145078611,
 234.79393621285774,
 150.92047108780764,
 167.215523378151

In [167]:
mean_VE_yellow = calc_mean_VE_yellow(df, start_row_patient, yellow_start, yellow_end)
mean_VE_yellow

[5.9258258719193311,
 7.4630940832742834,
 4.7677609125773115,
 5.6580311212784204,
 6.406623921743253,
 6.2797541220982884,
 1.5341875278032748,
 6.4591983037117204,
 6.7985440638007191,
 8.6643787360772855,
 5.4919517162518616,
 5.5307197745253402,
 6.1422797063501866,
 6.7390220804912273,
 5.7369481300821112,
 4.6846195255837788,
 5.33313386033221,
 6.710107954536995,
 11.35456418991091,
 6.2876350998878481,
 7.0122141428571423,
 6.0798301929380827,
 7.2433431439283416,
 8.5734576830049853,
 6.17003100092818,
 4.5877893785151045,
 7.1015457990692852,
 8.9791731136601136,
 6.3744550496339807,
 6.3512999216715498,
 9.7119494414911003,
 4.7228142082691189,
 5.1336764639074151,
 5.5772109603881832,
 6.4350474404125677,
 7.8077976471040289,
 6.1917799767993742,
 4.4402687424107601,
 6.0580190681829684,
 5.5784717652855846,
 7.5723291024929136,
 5.2886873902439024,
 4.7404899800696025,
 7.8799947670527866,
 7.559686032737174,
 7.5037699540456133,
 5.2609538123721169,
 5.1243225946658999,


In [168]:
mean_BPM_yellow = calc_mean_BPM_yellow(df, start_row_patient, yellow_start, yellow_end)
mean_BPM_yellow

[10.36474953199688,
 13.978803704424601,
 11.13993867238362,
 11.885886656932346,
 17.168000663199074,
 15.853991746902459,
 6.3622929499699508,
 12.654473402561292,
 16.805383531058709,
 16.068635475344774,
 6.1642805551871271,
 10.696025313400643,
 8.3012457126524382,
 17.780246595057044,
 14.043841770717084,
 12.526438061783956,
 12.217034619028976,
 16.209191973616441,
 7.2681894302368146,
 16.997217369079596,
 15.215928357142857,
 17.392989251671771,
 10.573774721564313,
 15.782917255308567,
 16.52943732098835,
 11.33766095231219,
 17.956775921147045,
 13.13627653587155,
 10.667201772332188,
 17.227141613430451,
 8.3598649211046112,
 4.6030895531177514,
 13.312929124543162,
 10.185457096099849,
 13.497895543168228,
 11.934812499255663,
 9.2074151266188835,
 11.929676244133395,
 12.700128904203092,
 13.668265528795201,
 12.731416725530863,
 7.1189373414634156,
 6.340708802386029,
 15.422593230292911,
 18.247216364232507,
 14.510655085245766,
 13.970523221152167,
 13.980970999089683

In [22]:
m, s = divmod(seconds, 60)
h, m = divmod(m, 60)
print "%d:%02d:%02d" % (h, m, s)

SyntaxError: invalid syntax (<ipython-input-22-5a7125a9dc4f>, line 3)

In [25]:
stat.mean([1,2,3,4,5])

3.0

In [43]:
value = df.iat[1, 11]
value

0.20146337151527399